In [ ]:
# Run these before starting jupyter
!conda activate feature_3dgs
!jupyter lab

In [12]:
%cd /home/songliyu/Documents/feature-3dgs
import os
ROOT = os.getcwd()
DATASET_NAME = '100MEDIA'
print(ROOT)
print(DATASET_NAME)

/home/songliyu/Documents/feature-3dgs
/home/songliyu/Documents/feature-3dgs
100MEDIA


In [4]:
%cd $ROOT

!mkdir data/$DATASET_NAME/input
!cp data/$DATASET_NAME/images/* data/$DATASET_NAME/input
!python convert.py -s data/$DATASET_NAME

/home/songliyu/Documents/feature-3dgs
I20260117 15:47:56.499715 3713818 misc.cc:198] 
Feature extraction
I20260117 15:47:56.825481 3713840 feature_extraction.cc:254] Processed file [1/138]
I20260117 15:47:56.825515 3713840 feature_extraction.cc:257]   Name:            DJI_0025.JPG
I20260117 15:47:56.825520 3713840 feature_extraction.cc:283]   Dimensions:      4032 x 3024
I20260117 15:47:56.825522 3713840 feature_extraction.cc:286]   Camera:          #1 - OPENCV
I20260117 15:47:56.825526 3713840 feature_extraction.cc:289]   Focal Length:    2764.80px (Prior)
I20260117 15:47:56.825532 3713840 feature_extraction.cc:296]   GPS:             LAT=37.526, LON=122.056, ALT=54.184
I20260117 15:47:56.825536 3713840 feature_extraction.cc:302]   Features:        8389
I20260117 15:47:58.040041 3713840 feature_extraction.cc:254] Processed file [2/138]
I20260117 15:47:58.040081 3713840 feature_extraction.cc:257]   Name:            DJI_0026.JPG
I20260117 15:47:58.040086 3713840 feature_extraction.cc:28

In [5]:
%cd $ROOT/encoders/sam_encoder
!python export_image_embeddings.py \
    --checkpoint checkpoints/sam_vit_h_4b8939.pth \
    --model-type vit_h \
    --input ../../data/$DATASET_NAME/images \
    --output ../../data/$DATASET_NAME/sam_embeddings

/home/songliyu/Documents/feature-3dgs/encoders/sam_encoder
Loading model...
Processing '../../data/100MEDIA/images/DJI_0091.JPG'...
Using features encoded from input image!
embedding shape:  torch.Size([256, 64, 64])
image_embedding_tensor_cropped:  torch.Size([256, 48, 64])
Processing '../../data/100MEDIA/images/DJI_0083.JPG'...
Using features encoded from input image!
embedding shape:  torch.Size([256, 64, 64])
image_embedding_tensor_cropped:  torch.Size([256, 48, 64])
Processing '../../data/100MEDIA/images/DJI_0140.JPG'...
Using features encoded from input image!
embedding shape:  torch.Size([256, 64, 64])
image_embedding_tensor_cropped:  torch.Size([256, 48, 64])
Processing '../../data/100MEDIA/images/DJI_0040.JPG'...
Using features encoded from input image!
embedding shape:  torch.Size([256, 64, 64])
image_embedding_tensor_cropped:  torch.Size([256, 48, 64])
Processing '../../data/100MEDIA/images/DJI_0157.JPG'...
Using features encoded from input image!
embedding shape:  torch.Siz

In [25]:
%cd $ROOT
!sed -i 's/#define NUM_SEMANTIC_CHANNELS 128 \/\/ Subject to change/#define NUM_SEMANTIC_CHANNELS 64 \/\/ Subject to change/' submodules/diff-gaussian-rasterization-feature/cuda_rasterizer/config.h
!rm -rf submodules/diff-gaussian-rasterization-feature/build
!pip install --no-build-isolation submodules/diff-gaussian-rasterization-feature
!python train.py -s data/south-building -m output/south-building--sam-7000 -f sam --speedup --iterations 7000
!python render.py -s data/south-building -m output/south-building--sam-7000 -f sam --iteration 7000 --skip_test --novel_view
%cd $ROOT/encoders/sam_encoder
!python segment.py --checkpoint checkpoints/sam_vit_h_4b8939.pth --model-type vit_h --data ../../output/south-building--sam-7000 --iteration 7000

/home/songliyu/Documents/feature-3dgs
Optimizing output/south-building--sam-7000
Output folder: output/south-building--sam-7000 [29/12 13:44:36]
Reading camera 128/128 [29/12 13:44:36]
Loading Training Cameras [29/12 13:44:36]
[ INFO ] Encountered quite large input images (>1.6K pixels width), rescaling to 1.6K.
 If this is not desired, please explicitly specify '--resolution/-r' as 1 [29/12 13:44:36]
Loading Test Cameras [29/12 13:44:48]
Number of points at initialisation :  81913 [29/12 13:44:48]
Training progress: 100%|██| 7000/7000 [1:57:30<00:00,  1.01s/it, Loss=0.1377521]

[ITER 7000] Evaluating train: L1 0.06010080724954606 PSNR 20.71012954711914 [29/12 15:42:23]

[ITER 7000] Saving Gaussians [29/12 15:42:23]

[ITER 7000] Saving feature decoder ckpt [29/12 15:42:26]

Training complete. [29/12 15:42:27]


In [8]:
%cd $ROOT
!sed -i 's/#define NUM_SEMANTIC_CHANNELS 128 \/\/ Subject to change/#define NUM_SEMANTIC_CHANNELS 64 \/\/ Subject to change/' submodules/diff-gaussian-rasterization-feature/cuda_rasterizer/config.h
!rm -rf submodules/diff-gaussian-rasterization-feature/build
!pip install --no-build-isolation submodules/diff-gaussian-rasterization-feature
!python train.py -s data/south-building -m output/south-building--sam-30000 -f sam --speedup --iterations 30000
!python render.py -s data/south-building -m output/south-building--sam-30000 -f sam --iteration 30000 --skip_test --novel_view
%cd encoders/sam_encoder
!python segment.py --checkpoint checkpoints/sam_vit_h_4b8939.pth --model-type vit_h --data ../../output/south-building--sam-30000 --iteration 30000
%cd $ROOT
!python videos.py --data output/south-building--sam-30000 --fps 10 -f sam --iteration 30000

/home/songliyu/Documents/feature-3dgs
Looking in indexes: https://mirrors.aliyun.com/pypi/simple
Processing ./submodules/diff-gaussian-rasterization-feature
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diff_gaussian_rasterization: filename=diff_gaussian_rasterization-0.0.0-cp39-cp39-linux_x86_64.whl size=761532 sha256=594c36e607b33e8bbdbce37956d375d88ac10398f303bc58001000577d2bb3f2
  Stored in directory: /home/songliyu/.cache/pip/wheels/c6/87/01/436ef4be68243da469dfeea93c69e37369b9995ed6f8bf7310
Successfully built diff_gaussian_rasterization
  Attempting uninstall: diff_gaussian_rasterization
    Found existing installation: diff_gaussian_rasterization 0.0.0
    Uninstalling diff_gaussian_rasterization-0.0.0:
      Successfully uninstalled diff_gaussian_rasterization-0.0.0
Optimizing output/south-building--sam-30000
Output folder: output/south-building--sam-30000 [01/01 10:15:25]
Reading camera 128/128 [01/01 10:15:26]
Loading Training Cameras [01/01 10:15:26]
[ I

In [14]:
%cd $ROOT
!python render.py -s data/south-building -m output/south-building--sam-30000 -f sam --iteration 7000 --skip_test --novel_view
%cd $ROOT/encoders/sam_encoder
!python segment.py --checkpoint checkpoints/sam_vit_h_4b8939.pth --model-type vit_h --data ../../output/south-building--sam-30000 --iteration 7000

/home/songliyu/Documents/feature-3dgs/encoders/sam_encoder
Loading model...
test do not exists!
Segment progress:   0%|                                 | 0/128 [00:00<?, ?it/s]Using locally loaded features!
/home/songliyu/Documents/feature-3dgs/encoders/sam_encoder/segment.py:124: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
Segment progress:  16%|███▊                    | 20/128 [00:57<04:50,  2.69s/it]Using locally loaded features!
/home/songliyu/Documents/feature-3dgs/encoders/sam_encoder/segment.py:120: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(10,10))
Segment progress: 100%|███████████████████████| 200/200 [08:23<00:00,  2.52s/it]


In [8]:
%cd $ROOT/encoders/lseg_encoder
!python -u encode_images.py \
    --backbone clip_vitl16_384 \
    --weights demo_e200.ckpt \
    --widehead \
    --no-scaleinv \
    --outdir ../../data/$DATASET_NAME/rgb_feature_langseg \
    --test-rgb-dir ../../data/$DATASET_NAME/images \
    --workers 0

/home/songliyu/Documents/feature-3dgs/encoders/lseg_encoder
Namespace(model='encnet', backbone='clip_vitl16_384', dataset='ignore', workers=0, base_size=520, crop_size=480, train_split='train', aux=False, se_loss=False, se_weight=0.2, batch_size=16, test_batch_size=16, no_cuda=False, seed=1, weights='demo_e200.ckpt', eval=False, export=None, acc_bn=False, test_val=False, no_val=False, module='lseg', data_path=None, scale_inv=False, widehead=True, widehead_hr=False, ignore_index=-1, label_src='default', jobname='default', strict=True, arch_option=0, block_depth=0, activation='lrelu', outdir='../../data/100MEDIA/rgb_feature_langseg', test_rgb_dir='../../data/100MEDIA/images', resize_max=1.25, cuda=True)
/home/songliyu/.vocal/miniforge3/envs/feature_3dgs_pure/lib/python3.9/site-packages/pytorch_lightning/utilities/migration/migration.py:208: You have multiple `ModelCheckpoint` callback states in this checkpoint, but we found state keys that would end up colliding with each other after an 

In [9]:
%cd $ROOT
# !sed -i 's/#define NUM_SEMANTIC_CHANNELS 512 \/\/ Subject to change/#define NUM_SEMANTIC_CHANNELS 128 \/\/ Subject to change/' submodules/diff-gaussian-rasterization-feature/cuda_rasterizer/config.h
!rm -rf submodules/diff-gaussian-rasterization-feature/build
!pip install --no-build-isolation submodules/diff-gaussian-rasterization-feature
!python train.py -s data/$DATASET_NAME -m output/$DATASET_NAME--lseg-30000 -f lseg --speedup --iterations 30000
!python render.py -s data/$DATASET_NAME -m output/$DATASET_NAME--lseg-30000 -f lseg --iteration 30000 --skip_test --novel_view
%cd encoders/lseg_encoder
!python -u segmentation.py --data ../../output/$DATASET_NAME--lseg-30000/ --iteration 30000
%cd $ROOT
!python videos.py --data output/$DATASET_NAME--lseg-30000 --fps 10 -f lseg --iteration 30000

/home/songliyu/Documents/feature-3dgs
Looking in indexes: https://mirrors.aliyun.com/pypi/simple
Processing ./submodules/diff-gaussian-rasterization-feature
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diff_gaussian_rasterization: filename=diff_gaussian_rasterization-0.0.0-cp39-cp39-linux_x86_64.whl size=768379 sha256=35a3d865487686f51209577427158e816bf6f52f0d06d7c6031c3cbedc7ca7fc
  Stored in directory: /home/songliyu/.cache/pip/wheels/c6/87/01/436ef4be68243da469dfeea93c69e37369b9995ed6f8bf7310
Successfully built diff_gaussian_rasterization
  Attempting uninstall: diff_gaussian_rasterization
    Found existing installation: diff_gaussian_rasterization 0.0.0
    Uninstalling diff_gaussian_rasterization-0.0.0:
      Successfully uninstalled diff_gaussian_rasterization-0.0.0
Optimizing output/100MEDIA--lseg-30000
Output folder: output/100MEDIA--lseg-30000 [17/01 16:35:38]
Reading camera 23/23 [17/01 16:35:41]
Converting point3d.bin to .ply, will happen only the firs

In [13]:
%cd $ROOT
!python render.py -s data/$DATASET_NAME -m output/$DATASET_NAME--lseg-30000 -f lseg --iteration 7000 --skip_test --novel_view
%cd encoders/lseg_encoder
!python -u segmentation.py --data ../../output/$DATASET_NAME--lseg-30000/ --iteration 7000
%cd $ROOT
!python videos.py --data output/$DATASET_NAME--lseg-30000 --fps 10 -f lseg --iteration 7000

/home/songliyu/Documents/feature-3dgs
Looking for config file in output/100MEDIA--lseg-30000/cfg_args
Config file found: output/100MEDIA--lseg-30000/cfg_args
Rendering output/100MEDIA--lseg-30000
Loading trained model at iteration 7000 [18/01 09:00:43]
Reading camera 23/23 [18/01 09:00:45]
Loading Training Cameras [18/01 09:00:45]
[ INFO ] Encountered quite large input images (>1.6K pixels width), rescaling to 1.6K.
 If this is not desired, please explicitly specify '--resolution/-r' as 1 [18/01 09:00:45]
Loading Test Cameras [18/01 09:00:48]
Rendering progress: 100%|█████████████████████| 200/200 [05:28<00:00,  1.64s/it]
/home/songliyu/Documents/feature-3dgs/encoders/lseg_encoder
Namespace(model='encnet', backbone='clip_vitl16_384', dataset='ignore', workers=0, base_size=520, crop_size=480, train_split='train', aux=False, se_loss=False, se_weight=0.2, batch_size=16, test_batch_size=16, no_cuda=False, seed=1, weights='demo_e200.ckpt', eval=False, export=None, acc_bn=False, test_val=Fal

In [23]:
!ffmpeg -framerate 10 -i $ROOT/output/$DATASET_NAME--lseg-30000/novel_views/ours_7000/feature_map/%05d_feature_vis.png $ROOT/output/$DATASET_NAME--lseg-30000/videos_7000/novel_views_feature_vis.mp4
!ffmpeg -framerate 10 -i $ROOT/output/$DATASET_NAME--lseg-30000/novel_views/ours_7000/renders/%05d.png $ROOT/output/$DATASET_NAME--lseg-30000/videos_7000/novel_views_renders.mp4
!ffmpeg -framerate 10 -i $ROOT/output/$DATASET_NAME--lseg-30000/seg_7000_labels/novel_views/%05d_fmap_CxHxW.png_legend.png $ROOT/output/$DATASET_NAME--lseg-30000/videos_7000/novel_views_fmap_CxHxW.png_legend.mp4
!ffmpeg -framerate 10 -i $ROOT/output/$DATASET_NAME--lseg-30000/seg_7000_labels/novel_views/%05d_fmap_CxHxW.png_vis.png $ROOT/output/$DATASET_NAME--lseg-30000/videos_7000/novel_views_fmap_CxHxW.png_vis.mp4
!ffmpeg -framerate 10 -i $ROOT/output/$DATASET_NAME--lseg-30000/seg_7000_labels/train/%05d_fmap_CxHxW.png_legend.png $ROOT/output/$DATASET_NAME--lseg-30000/videos_7000/train_fmap_CxHxW.png_legend.mp4
!ffmpeg -framerate 10 -i $ROOT/output/$DATASET_NAME--lseg-30000/seg_7000_labels/train/%05d_fmap_CxHxW.png_vis.png $ROOT/output/$DATASET_NAME--lseg-30000/videos_7000/train_fmap_CxHxW.png_vis.mp4
!ffmpeg -framerate 10 -i $ROOT/output/$DATASET_NAME--lseg-30000/train/ours_7000/depth/%05d.png $ROOT/output/$DATASET_NAME--lseg-30000/videos_7000/train_depth.mp4
!ffmpeg -framerate 10 -i $ROOT/output/$DATASET_NAME--lseg-30000/train/ours_7000/feature_map/%05d_feature_vis.png $ROOT/output/$DATASET_NAME--lseg-30000/videos_7000/train_feature_vis.mp4
!ffmpeg -framerate 10 -i $ROOT/output/$DATASET_NAME--lseg-30000/train/ours_7000/gt/%05d.png $ROOT/output/$DATASET_NAME--lseg-30000/videos_7000/train_gt.mp4
!ffmpeg -framerate 10 -i $ROOT/output/$DATASET_NAME--lseg-30000/train/ours_7000/gt_feature_map/%05d_feature_vis.png $ROOT/output/$DATASET_NAME--lseg-30000/videos_7000/train_gt_feature_vis.mp4
!ffmpeg -framerate 10 -i $ROOT/output/$DATASET_NAME--lseg-30000/train/ours_7000/renders/%05d.png $ROOT/output/$DATASET_NAME--lseg-30000/videos_7000/train_renders.mp4

ffmpeg version 8.0.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 12 (Ubuntu 12.3.0-1ubuntu1~22.04.2)
  configuration: --prefix=/home/linuxbrew/.linuxbrew/Cellar/ffmpeg/8.0.1_1 --enable-shared --enable-pthreads --enable-version3 --cc=gcc-12 --host-cflags= --host-ldflags= --enable-ffplay --enable-gpl --enable-libsvtav1 --enable-libopus --enable-libx264 --enable-libmp3lame --enable-libdav1d --enable-libvpx --enable-libx265
  libavutil      60.  8.100 / 60.  8.100
  libavcodec     62. 11.100 / 62. 11.100
  libavformat    62.  3.100 / 62.  3.100
  libavdevice    62.  1.100 / 62.  1.100
  libavfilter    11.  4.100 / 11.  4.100
  libswscale      9.  1.100 /  9.  1.100
  libswresample   6.  1.100 /  6.  1.100
Input #0, image2, from '/home/songliyu/Documents/feature-3dgs/output/100MEDIA--lseg-30000/train/ours_7000/renders/%05d.png':
  Duration: 00:00:02.30, start: 0.000000, bitrate: N/A
  Stream #0:0: Video: png, rgb24(pc, gbr/unknown/unknown), 1600x1200, 10 fps, 10 tbr, 10 t